# ABR Standardization Notebook
---
---
*ABR remote sensing data, not land survey data*

# Table of contents
1. [PRE-PROCESSING](#preprocess)
    1. [Load fcover data](#loadfcover)
    2. [Join fcover to species checklist table](#joinchecklist)
    3. [Join fcover to leaf retention table](#joinleafretention)
    4. [Export species-PFT tables](#exporthabits)
    5. [Clean exported species-PFT tables](#cleanhabits)
2. [STANDARDIZATION](#standardization)
    1. [Standardize PFT names](#pftnames)
    2. [Sum species fcover to PFT-level](#sumfcover)
    3. [Standardize table layout](#tablelayout)
    4. [Standardize auxiliary columns](#aux)
        1. [Basic plot information](#basics)
        2. [Geographical intersections](#geo)
        3. [Duplicate coordinates](#dupes)
3. [EXPORT](#export)

# Load required packages

In [58]:
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pyogrio import read_dataframe
import glob
import geopandas as gpd
import sys
import os
import chardet
import tarfile
from urllib.request import urlretrieve
import regex as re
from shapely.validation import make_valid

In [59]:
# import local functions
current_dir = os.path.abspath('..')
if current_dir not in sys.path:
    sys.path.append(current_dir)
import standardize_pft_funcs as spf

---
---
# 1. Pre-processing <a name="preprocess"></a>
After visually inspecting each species-level fcover table for obvious errors, we saved the "cleaned" tables to a folder that we then standardized here.

---
## 1.1. Load fcover data <a name="loadfcover"></a>

In [60]:
source = 'abr'

In [61]:
# read vegetation fcover as dataframe
fcover_path = f'input_data/abr_foliar_cover.csv'
fcover = pd.read_csv(fcover_path)
fcover = fcover[['plot_id', 'veg_taxonomy', 'species_cover']]

In [62]:
# drop plots that start with a letter because they are ELS plots
startswith_digit = [x.isdigit() for x in fcover['plot_id'].str[0]]
fcover['plot_sw_digit'] = startswith_digit
fcover = fcover[fcover['plot_sw_digit'] == True]
fcover.head(3)

,plot_id,veg_taxonomy,species_cover,plot_sw_digit
0,120712b1,Litter,18.54,True
1,120712b1,Hylocomium splendens,15.89,True
2,120712b1,Carex bigelowii,11.26,True


In [63]:
# get unique species names from narrowed down list
u_species_names = fcover['veg_taxonomy'].unique()
print(len(u_species_names))
u_species_names = [x for x in u_species_names if pd.notnull(x)]
print(len(u_species_names))
species_names_df = pd.DataFrame(u_species_names, columns=['datasetSpeciesName'])

385
385


---
## 1.2. Join fcover to species checklist table <a name="joinchecklist"></a>

In [64]:
# load species checklist and prep for joining
checklist_path = f'../etc/akveg_species_checklist.csv'
checklist_df = spf.checklist_df(checklist_path)
checklist_df.head(3)

,nameCode,checklistSpeciesName,nameStatus,nameAccepted,nameFamily,acceptedNameSource,nameLevel,speciesForm,speciesHabit
0,abroth,Abrothallus De Not.,accepted,Abrothallus De Not.,Abrothallaceae,CNALH,genus,lichen,lichen
1,abrpar,Abrothallus parmeliarum (Sommerf.) Arnold,accepted,Abrothallus parmeliarum (Sommerf.) Arnold,Abrothallaceae,CNALH,species,lichen,lichen
2,abrpey,Abrothallus peyritschii (Stein) Kotte,accepted,Abrothallus peyritschii (Stein) Kotte,Abrothallaceae,Dillman et al. 2012,species,lichen,lichen


In [65]:
species_names_df.head(3)

,datasetSpeciesName
0,Litter
1,Hylocomium splendens
2,Carex bigelowii


In [66]:
# get first 2 words (genus-species) from checklist accepted name and data species name
checklist_df['joinKey'] = checklist_df['checklistSpeciesName'].apply(spf.get_substrings)
species_names_df['joinKey'] = species_names_df['datasetSpeciesName'].apply(spf.get_substrings)
habits = spf.join_to_checklist(unique_species=species_names_df, 
                               checklist=checklist_df, 
                               u_name='datasetSpeciesName', 
                               c_unofficial_name='checklistSpeciesName', 
                               c_official_name='nameAccepted', 
                               mapping_name='joinKey',
                               habit='speciesHabit')
habits.head(3)

161 species are missing habits.
93 species still missing habits.
27 species still missing habits.
25 species still missing habits.


,datasetSpeciesName,joinKey,speciesHabit
index,,,
0,Litter,Litter,NaN
1,Hylocomium splendens,Hylocomium splendens,moss
2,Carex bigelowii,Carex bigelowii,graminoid


In [67]:
species_names_df.head(3)

,datasetSpeciesName,joinKey
0,Litter,Litter
1,Hylocomium splendens,Hylocomium splendens
2,Carex bigelowii,Carex bigelowii


In [68]:
habits = species_names_df.merge(habits, how='left', left_on='datasetSpeciesName', right_on='datasetSpeciesName', suffixes=(None, '_1'))
habits = habits[['joinKey', 'datasetSpeciesName', 'speciesHabit']]

---
## 1.3. Join fcover to leaf retention table <a name="joinleafretention"></a>

In [69]:
leaf_df = spf.leaf_retention_df('../etc/macander_leaf_retention.csv')
leaf_df.head(3)

,leafRetention,retentionSpeciesName
0,deciduous,Salix
1,deciduous,Betula nana
2,deciduous,Vaccinium uliginosum


In [70]:
# add leaf retention column
habits_wleaf = spf.add_leaf_retention(habits, leaf_df, 'leafRetention')
habits_wleaf = habits_wleaf[['datasetSpeciesName', 'joinKey', 'speciesHabit', 'leafRetention']]
habits_wleaf.head(3)

,datasetSpeciesName,joinKey,speciesHabit,leafRetention
0,Litter,Litter,NaN,
1,Hylocomium splendens,Hylocomium splendens,moss,
2,Carex bigelowii,Carex bigelowii,graminoid,


In [71]:
# apply function
habits_wleaf['speciesHabit'] = habits_wleaf['speciesHabit'].apply(spf.clean_shrub_habits)
list(habits_wleaf['speciesHabit'].unique())

[nan,
 'moss',
 'graminoid',
 'shrub',
 'forb',
 'lichen',
 'spore-bearing',
 'liverwort']

---
## 1.4. Export species-PFT tables <a name="exporthabits"></a>

In [72]:
# generate 3 files and dataframe of habit types
shrubs, nonshrubs, null = spf.export_habit_files(habits_wleaf,
                                                 'temp_data', 
                                                 source, 
                                                 'speciesHabit')

In [73]:
# check to make sure the files make sense
len(habits_wleaf) == (len(null) + len(nonshrubs) + len(shrubs))

True

---
## 1.5. Clean exported species-PFT tables <a name="cleanhabits"></a>
Here, we manually assessed each file. The nonshrubs were usually correct and did not have to be adjusted. Some shrubs were missing leaf retention information, so we consulted with arctic ecologists to determine it. For unknown (null) habits, we consulted arctic ecology expert, Amy Breen, to determine the issue and correct habit. After the tables were complete, all species names had a single associated habit.
- null habits
    - assign missing habits and consult ecologist when needed
- non-shrubs
    - assign leaf habit to all trees (ctrl-f to find all trees in excel)
        - coniferous tree = evergreen
        - deciduous tree  = deciduous
    - remove leaf habit if misapplied
- shrubs
    - check current leaf habits and correct if necessary
    - fill empty leaf habits
    - choose leaf habit if both evergreen & deciduous listed
    - add Stature column (dwarf shrub or not)

---
---
# 2. Standardization <a name="standardization"></a>

---
## 2.1. Standardize PFT names <a name="pftnames"></a>
deciduous shrub, evergreen shrub, graminoid, forb, litter, non-vascular (lichen, brophyte)

In [74]:
# paths to visually assessed and cleaned habit files
all_habit_types = [f'temp_data/{source}_nonshrubs_01.csv',
                   f'temp_data/{source}_shrubs_01.csv',
                   f'temp_data/{source}_nullhabit_01.csv']

# create list of habit dataframes
habit_dfs = []
for path in all_habit_types:
    habit_df = pd.read_csv(path, index_col=0)
    habit_dfs.append(habit_df)
all_habits = pd.concat(habit_dfs)

In [75]:
# show unique habit names we're working with
all_habits['speciesHabit'].unique().tolist()

['forb',
 'graminoid',
 'lichen',
 'liverwort',
 'moss',
 'spore-bearing',
 'shrub',
 'bare ground',
 'bare mineral',
 'crust',
 'litter',
 'scat',
 'algae',
 'fungus',
 'water']

In [76]:
# akveg-specific habit name standardization function
def standardize_habit(habitrow, leafrow):
    
    habitrow = habitrow.to_numpy()
    leafrow = leafrow.to_numpy()
    new_row = []
    
    for habit, leaf in zip(habitrow, leafrow):
        
        habit = habit.lower()
        # non-shrubs
        if 'algae' in habit:
            new_habit = 'other'
        elif 'moss' in habit or 'liverwort' in habit:
            new_habit = 'bryophyte'
        elif 'spore-bearing' in habit:
            new_habit = 'forb'
        elif 'litter' in habit or 'scat' in habit:
            new_habit = 'litter'
        elif 'bare ground' in habit or 'crust' in habit or 'mineral' in habit:
            new_habit = 'bare ground'
            
        # shrubs and trees
        elif 'shrub' in habit:
            new_habit = f'{leaf} shrub'
            
        else:
            new_habit = f'{habit}'
        new_row.append(new_habit)
    
    return new_row

In [77]:
# standardized PFT
standard_habit_col = standardize_habit(all_habits['speciesHabit'], 
                                       all_habits['leafRetention'])
all_habits['standardHabit'] = standard_habit_col
all_habits.rename(columns={'speciesHabit':'nonstandardHabit'}, inplace=True)
print(len(all_habits))
all_habits.head(3)

385


,datasetSpeciesName,joinKey,nonstandardHabit,leafRetention,speciesStature,standardHabit
301,Achillea millefolium,Achillea millefolium,forb,NaN,NaN,forb
298,Agropyron sp.,Agropyron sp.,graminoid,NaN,NaN,graminoid
168,Alectoria nigricans,Alectoria nigricans,lichen,NaN,NaN,lichen


---
## 2.2. Sum species fcover to PFT-level <a name="sumfcover"></a>

In [78]:
fcover

,plot_id,veg_taxonomy,species_cover,plot_sw_digit
0,120712b1,Litter,18.54,True
1,120712b1,Hylocomium splendens,15.89,True
2,120712b1,Carex bigelowii,11.26,True
3,120712b1,Alnus fruticosa,10.60,True
4,120712b1,Vaccinium uliginosum,9.93,True
...,...,...,...,...
2756,120815b1,Eriophorum angustifolium ssp. triste,2.27,True
2757,120815b1,Salix pulchra,2.27,True
2758,120815b1,Cladonia gracilis,2.27,True
2759,120815b1,Calamagrostis sp.,2.27,True


In [79]:
# loop through each plot and sum fcover per pft
all_habits = all_habits[['datasetSpeciesName', 'standardHabit', 'nonstandardHabit']]
species_fcover = fcover.merge(all_habits,
                              left_on='veg_taxonomy',
                              right_on='datasetSpeciesName')
species_fcover = species_fcover.rename(columns={'species_cover':'percentCover', 'plot_id':'plotName'})
species_fcover = species_fcover.drop(columns=['veg_taxonomy','plot_sw_digit'])
species_fcover.head(3)

,plotName,percentCover,datasetSpeciesName,standardHabit,nonstandardHabit
0,120712b1,18.54,Litter,litter,litter
1,120713a1,77.78,Litter,litter,litter
2,120713b1,32.67,Litter,litter,litter


In [80]:
species_fcover = species_fcover[species_fcover['percentCover'].notna() & (species_fcover['percentCover'] != 0)]
species_fcover = species_fcover[['plotName', 'datasetSpeciesName', 'standardHabit', 
                                 'nonstandardHabit', 'percentCover']]
species_fcover['standardHabit'] = species_fcover['standardHabit'].replace('fungus', 'other')
species_fcover.head(3)

,plotName,datasetSpeciesName,standardHabit,nonstandardHabit,percentCover
0,120712b1,Litter,litter,litter,18.54
1,120713a1,Litter,litter,litter,77.78
2,120713b1,Litter,litter,litter,32.67


In [81]:
# export
species_fcover.to_csv(f'output_data/{source}_nonstandard_species_fcover.csv', index=False)

In [82]:
# group by 'Site Code' and 'standardHabit', then sum the 'Cover'
temp = species_fcover.groupby(['plotName', 'standardHabit'])['percentCover'].sum().reset_index()
pft_fcover = temp.pivot(index='plotName', columns='standardHabit', values='percentCover')
pft_fcover = pft_fcover.fillna(0)

In [83]:
# clean up
pft_fcover.columns.name = None
pft_fcover.index.name = 'plotName'
pft_fcover.index = pft_fcover.index.astype(str)
pft_fcover.head(3)

,bare ground,bryophyte,deciduous shrub,evergreen shrub,forb,graminoid,lichen,litter,other,water
plotName,,,,,,,,,,
120712b1,0.00,31.77,32.45,20.52,2.64,19.20,1.32,18.54,0.0,0.0
120713a1,28.76,36.60,0.00,0.00,1.96,28.75,0.65,77.78,0.0,0.0
120713b1,0.00,17.34,11.34,38.01,0.00,14.66,8.66,32.67,0.0,0.0


In [84]:
# drop water and bare ground total cover
# we want top cover, which is in the nonveg df; these are currently total cover
pft_fcover = pft_fcover.drop(columns=['bare ground', 'water'])

---
## 2.3. Standardize table layout <a name="tablelayout"></a>

In [85]:
# list of necessary PFTs
pfts = ['deciduous shrub', 'deciduous tree',
        'evergreen shrub', 'evergreen tree',
        'forb', 'graminoid', 'non-vascular',
        'bryophyte', 'lichen', 'litter', 'other']

In [86]:
# load non-vegetation data and select
nonveg_path = f'input_data/abr_nonfoliar_cover.csv'
nonveg = pd.read_csv(nonveg_path)
nonveg = nonveg[['plot_id', 'water_topcov', 'bareground_topcov']]
nonveg = nonveg.rename(columns=dict(plot_id='plotName',
                                    water_topcov='water',
                                    bareground_topcov='bare ground'))
nonveg = nonveg.set_index('plotName')
nonveg.head(3)

,water,bare ground
plotName,,
120716b2,0.0,0.0
120717a1,14.0,0.0
120718a1,0.0,0.0


In [87]:
# add non-vegetation information
pft_fcover = pft_fcover.merge(nonveg,
                              how='left',
                              left_index=True,
                              right_index=True)

In [88]:
# create non-vascular
pft_fcover['non-vascular'] = pft_fcover['bryophyte'] + pft_fcover['lichen']

In [89]:
# create standard columns
pft_fcover = spf.add_standard_cols(pft_fcover, pfts)
pft_fcover = pft_fcover[pfts]
pft_fcover = pft_fcover.merge(nonveg, how='left', left_index=True, right_index=True)
pft_fcover.head(3)

,deciduous shrub,deciduous tree,evergreen shrub,evergreen tree,forb,graminoid,non-vascular,bryophyte,lichen,litter,other,water,bare ground
plotName,,,,,,,,,,,,,
120712b1,32.45,NaN,20.52,NaN,2.64,19.20,33.09,31.77,1.32,18.54,0.0,0.0,0.00
120713a1,0.00,NaN,0.00,NaN,1.96,28.75,37.25,36.60,0.65,77.78,0.0,0.0,11.11
120713b1,11.34,NaN,38.01,NaN,0.00,14.66,26.00,17.34,8.66,32.67,0.0,0.0,0.00


---
## 2.4. Standardize auxiliary columns <a name="aux"></a>

Plot information:
- [ ] UID
- [x] plotName
- [x] dataSource
- [x] dataSubsource
- [x] fcoverScale
- [x] surveyMethod
- [x] surveyPurpose
- [x] duplicatedCoords
- [x] duplicatedDate
- [x] surveyYear
- [x] surveyMonth
- [x] surveyDay

Geographical information:
- [x] bioclimateSubzone
- [x] adminCountry
- [x] adminUnit
- [x] fireYears
- [x] coordEpsg
- [x] georefSource
- [x] georefAccuracy
- [x] plotRadius
- [x] longitueX
- [x] latitudeY

In [90]:
# load auxiliary data
file = f'input_data/abr_ancillary.csv'
aux = pd.read_csv(file)
# sampling methods, coverscale, data source
methods = pd.read_csv('../etc/sampling_methods.csv')

### 2.4.1. Basic plot information <a name="basics"></a>

In [91]:
new_aux = pd.DataFrame()

# date columns
aux['surveyDate'] = pd.to_datetime(aux['field_start_ts'], format='mixed')
new_aux['surveyYear'] = aux['surveyDate'].dt.year
new_aux['surveyMonth'] = aux['surveyDate'].dt.month
new_aux['surveyDay'] = aux['surveyDate'].dt.day

# plot size
new_aux['plotRadius'] = 55

# geographical information
new_aux['latitudeY'] = aux['latitude'].astype('Float32')
new_aux['longitudeX'] = aux['longitude'].astype('Float32')
new_aux['georefSource'] = 'GPS'
new_aux['georefAccuracy'] = np.nan
new_aux['coordEPSG'] = 'EPSG:4326' # will need to convert from EPSG:4269

# main
new_aux['plotName'] = aux['plot_id']
new_aux['dataSubsource'] = 'Shell Onshore Macander (pub 2017)'
new_aux = new_aux.merge(methods, how='left', left_on='dataSubsource', right_on='dataSubsource')
new_aux = new_aux.set_index(new_aux['plotName'])

### 2.4.2. Geographical intersections <a name="geo"></a>

In [92]:
# set geo info
geo_paths = ['../etc/gaul1/gaul1_asap.shp',
             '../etc/fire/InterAgencyFirePerimeterHistory_All_Years_View.shp',
             '../etc/bioclim/bioclimate_la_latlon.shp']
geo_names = ['gaul1',
             'fire',
             'bioclim']
geo_cols = [['name1', 'name0', 'geometry'],
            ['FIRE_YEAR', 'geometry'],
            ['zone', 'geometry']]

In [93]:
# create geospatial information
fcover_epsg = 'EPSG:4326' # wgs84
intersect_epsg = 'EPSG:5936' # alaska projected datum
fcover_and_aux = pd.concat([pft_fcover, new_aux], join='inner', axis=1)
fcover_and_aux = gpd.GeoDataFrame(fcover_and_aux, 
                                  geometry=gpd.points_from_xy(fcover_and_aux['longitudeX'],
                                                              fcover_and_aux['latitudeY']),
                                  crs=fcover_epsg)
fcover_and_aux = fcover_and_aux.to_crs(intersect_epsg)

# add new geospatial intersections
fcover_and_aux = spf.add_geospatial_aux(fcover_and_aux, 
                                        geo_paths, 
                                        geo_names, 
                                        geo_cols,
                                        intersect_epsg)

# drop unneccesary columns
fcover_and_aux = fcover_and_aux.drop(columns=['index_gaul1', 
                                              'index_fire', 
                                              'index_bioclim', 
                                              'plotName'])

In [94]:
# rename new columns
fcover_and_aux.rename(columns={'name1':'adminUnit',
                               'name0':'adminCountry',
                               'FIRE_YEAR':'fireYears',
                               'zone':'bioclimSubzone'}, inplace=True)

### 2.4.3. Duplicate coordinates <a name="dupes"></a>
This indicates that plots may have been revisited, they were very close and GPS accuracy was too low, or there is human error

In [95]:
# duplicate coord column
coords = ['longitudeX', 'latitudeY']
fcover_and_aux['duplicatedCoords'] = np.nan
fcover_and_aux = spf.find_duplicates(fcover_and_aux, coords, 'duplicatedCoords')

# duplicate date column
date = ['surveyYear', 'surveyMonth', 'surveyDay']
fcover_and_aux['duplicatedDate'] = np.nan
fcover_and_aux = spf.find_duplicates(fcover_and_aux, date, 'duplicatedDate')

duplicates found
duplicates found


---
# 3. Export <a name="export"></a>

In [96]:
# clean up
pfts = {'deciduous shrub':'deciduousShrubCover',
        'evergreen shrub':'evergreenShrubCover',
        'deciduous tree':'deciduousTreeCover',
        'evergreen tree':'evergreenTreeCover',
        'forb':'forbCover',
        'graminoid':'graminoidCover',
        'non-vascular':'nonvascularSumCover',
        'bryophyte':'bryophyteCover',
        'lichen':'lichenCover',
        'litter':'litterCover',
        'bare ground':'baregroundCover',
        'water':'waterCover',
        'other':'otherCover'}

# rename columns and replace NaN cover with 0
fcover_and_aux.rename(columns=pfts, inplace=True)
fcover_cols = list(pfts.values())
fcover_and_aux[fcover_cols] = fcover_and_aux[fcover_cols].fillna(0.0)

# reproject and set cover data type
fcover_and_aux = fcover_and_aux.to_crs('EPSG:4326')
fcover_and_aux[fcover_cols] = fcover_and_aux[fcover_cols].astype(np.float32)
fcover_and_aux.index.name = 'plotName'

In [97]:
fcover_and_aux.columns

Index(['deciduousShrubCover', 'deciduousTreeCover', 'evergreenShrubCover',
       'evergreenTreeCover', 'forbCover', 'graminoidCover',
       'nonvascularSumCover', 'bryophyteCover', 'lichenCover', 'litterCover',
       'otherCover', 'waterCover', 'baregroundCover', 'surveyYear',
       'surveyMonth', 'surveyDay', 'plotRadius', 'latitudeY', 'longitudeX',
       'georefSource', 'georefAccuracy', 'coordEPSG', 'dataSubsource',
       'dataSource', 'dataSourceHtml', 'dataSubsourceCitation', 'surveyMethod',
       'fcoverScale', 'surveyPurpose', 'geometry', 'adminUnit', 'adminCountry',
       'fireYears', 'bioclimSubzone', 'duplicatedCoords', 'duplicatedDate'],
      dtype='object')

In [98]:
# Export cover data
covercols = [col for col in fcover_and_aux.columns if 'Cover' in col]
auxcols = [col for col in fcover_and_aux.columns if 'Cover' not in col]
pft_fcover = fcover_and_aux[covercols]
pft_fcover.to_csv(f'output_data/{source}_standard_pft_fcover.csv')

# Export aux data
pft_aux = fcover_and_aux[auxcols]
pft_aux.to_csv(f'output_data/{source}_plot_info.csv')